In [1]:
import awkward as ak
import dask
import dask_awkward
from distributed import performance_report
import hist
from hist.dask import Hist
import json
from coffea import processor
from coffea.nanoevents import BaseSchema, PFNanoAODSchema, NanoAODSchema
from coffea.dataset_tools import apply_to_dataset, apply_to_fileset, preprocess
from coffea.dataset_tools import max_chunks, max_files, slice_chunks, slice_files
from coffea.analysis_tools import Weights, PackedSelection
import matplotlib.pyplot as plt
from coffea.nanoevents.methods import vector
from coffea.nanoevents import NanoEventsFactory

/tmp/ipykernel_2182163/3704368154.py:14: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:
import fsspec_xrootd
import dask.multiprocessing
import os

In [3]:
# import uproot
# url = "root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_20.root"
# uproot.open(url)["Events"].show()

In [4]:
# import XRootD.client


In [5]:
from msdprocessor import msdProcessor

In [6]:
# from dask.distributed import Client
# # client = Client("tls://localhost:8786")

# client = Client(n_workers = 10)
# client

In [7]:
os.environ["CONDOR_CONFIG"] = ".condor_config"

In [8]:
! export CONDOR_CONFIG=".condor_config"

In [9]:
! export VIRTUAL_ENV="/uscms/home/nyabando/nobackup/miniconda3/envs/fastjet"

In [10]:
from distributed import Client
from lpcjobqueue import LPCCondorCluster


Condor configuration not found! run the following command outside the apptainer instance
grep -v '^include' /etc/condor/config.d/01_cmslpc_interactive > .condor_config
File specified in CONDOR_CONFIG environment variable:
"/srv/.condor_config" does not exist.

Neither the environment variable CONDOR_CONFIG,
/etc/condor/, /usr/local/etc/, nor ~condor/ contain a condor_config source.
Either set CONDOR_CONFIG to point to a valid config source,
or put a "condor_config" file in /etc/condor/ /usr/local/etc/ or ~condor/


In [11]:
# env_extra = [
#     f"export PYTHONPATH=$PYTHONPATH:{os.getcwd()}"]

In [12]:
# cluster = LPCCondorCluster(
#     job_script_prologue=[
#         "export DASK_DISTRIBUTED__WORKER__PROFILE__INTERVAL=1d",
#         "export DASK_DISTRIBUTED__WORKER__PROFILE__CYCLE=2d",
#     ],
#     shared_temp_directory="/tmp",
#     transfer_input_files=["msdprocessor.py"],
#     ship_env=True,
#     memory="4GB",
# #    image="coffeateam/coffea-dask:0.7.20-fastjet-3.4.0.1-g4cab023"
# )

In [13]:
# cluster= LocalCluster(memory_limit='8GB')
# cluster.scale(10)
# cluster.adapt(minimum=1, maximum=20)
client= Client()
client

/uscms/home/nyabando/nobackup/miniconda3/envs/fastjet/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33357 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/33357/status,
Dashboard: /proxy/33357/status,Workers: 4
Total threads: 8,Total memory: 11.43 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35929,Workers: 4
Dashboard: /proxy/33357/status,Total threads: 8
Started: Just now,Total memory: 11.43 GiB
Comm: tcp://127.0.0.1:42089,Total threads: 2
Dashboard: /proxy/38279/status,Memory: 2.86 GiB
Nanny: tcp://127.0.0.1:42845,


In [14]:
# with open("2018_GluGluHToBB.json", "rt") as file:
#     initial_fileset = json.load(file)

In [15]:
# import uproot
# initial_fileset = "root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_20.root"
# uproot.open(initial_fileset)["Events"]

In [16]:
with open("2018_QCD_Pt1000to1400.json", "rt") as file:
    initial_fileset = json.load(file)

In [17]:
# print(initial_fileset)

In [18]:
files={}
for i, j in initial_fileset.items():
    print(f"this is i {i}")
    files[i]={"files":{}}
    for k in j:
        # k = k.replace("cmsxrootd.fnal.gov", "xcache")
        files[i]["files"][k] = "Events"
        print(f"this is k {k}")
    

this is i QCD_Pt_1000to1400
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/QCD_Pt_1000to1400/211111_021521/0000/nano_mc2018_1-1.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/QCD_Pt_1000to1400/211111_021521/0000/nano_mc2018_1-10.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/QCD_Pt_1000to1400/211111_021521/0000/nano_mc2018_1-11.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/QCD_Pt_1000to1400/211111_021521/0000/nano_mc2018_1-12.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/QCD_Pt_1000to1400/211111_021521/0000/nano_mc2018_1-13.root
this is k root://cmsxrootd.fnal.gov//store/user/lpcpfnano/jekrupa/v2_2/2018/QC

In [19]:
# print(files)

In [20]:
# class Client:
#     def restart(self):
#         print("Restarting client...")

# client_instance = Client() 
# client_instance.restart()

In [21]:
preprocessed_available, preprocessed_total = preprocess(
        files,
        step_size=50_000,
        align_clusters=None,
        skip_bad_files=True,
        recalculate_steps=False,
        files_per_batch=1,
        file_exceptions=(OSError,),
        save_form=True,
        uproot_options={},
        step_size_safety_factor=0.5,
    )

2024-07-31 12:19:42,812 - distributed.utils_perf - WARNING - full garbage collections took 39% CPU time recently (threshold: 10%)
2024-07-31 12:19:44,349 - distributed.utils_perf - WARNING - full garbage collections took 39% CPU time recently (threshold: 10%)
2024-07-31 12:19:45,109 - distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
2024-07-31 12:19:45,897 - distributed.utils_perf - WARNING - full garbage collections took 39% CPU time recently (threshold: 10%)
2024-07-31 12:19:46,540 - distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
2024-07-31 12:19:46,975 - distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
2024-07-31 12:19:47,089 - distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
2024-07-31 12:19:47,202 - distributed.utils_perf - WARNING - full garbage collections took

In [22]:
print(preprocessed_available.keys())

dict_keys(['QCD_Pt_1000to1400'])


In [23]:
import gzip, pickle, json
output_file = "scaleout_fileset"
with gzip.open(f"{output_file}_available.json.gz", "wt") as file:
    json.dump(preprocessed_available, file, indent=2)
    print(f"Saved available fileset chunks to {output_file}_available.json.gz")
with gzip.open(f"{output_file}_all.json.gz", "wt") as file:
    json.dump(preprocessed_total, file, indent=2)
    print(f"Saved complete fileset chunks to {output_file}_all.json.gz")

Saved available fileset chunks to scaleout_fileset_available.json.gz
Saved complete fileset chunks to scaleout_fileset_all.json.gz


In [34]:
test_preprocessed_files = max_files(preprocessed_available, 10)
test_preprocessed = max_chunks(test_preprocessed_files, 30)

In [35]:
print(test_preprocessed_files)

{'QCD_Pt_1000to1400': {'files': {'root://cmsxrootd.fnal.gov:1094//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/QCD_Pt_1000to1400/211111_021521/0000/nano_mc2018_1-1.root': {'object_path': 'Events', 'steps': [[0, 22000]], 'num_entries': 22000, 'uuid': '2bc141a2-43df-11ec-aef7-c926bb94beef'}, 'root://cmsxrootd.fnal.gov:1094//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/QCD_Pt_1000to1400/211111_021521/0000/nano_mc2018_1-10.root': {'object_path': 'Events', 'steps': [[0, 22000]], 'num_entries': 22000, 'uuid': '2d3e22de-43df-11ec-805a-7527bb94beef'}, 'root://cmsxrootd.fnal.gov:1094//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/QCD_Pt_1000to1400/211111_021521/0000/nano_mc2018_1-11.root': {'object_path': 'Events', 'steps': [[0, 22000]], 'num_entries': 22000, 'uuid': '50d1c90c-43e0-11ec-91b3-7527bb94beef'}, 'root://cmsxrootd.fnal.gov:1094//store/user/lpcpfnano/jekrupa/v2_2/2018/QCD/Q

In [36]:
compute, results = apply_to_fileset(data_manipulation=msdProcessor(),    
                            fileset=test_preprocessed_files,
                            schemaclass=PFNanoAODSchema,
                            uproot_options={"allow_read_errors_with_report": (OSError, KeyError)},
                           )

In [37]:
compute_results=dask.compute(compute)

2024-07-31 12:24:57,577 - distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
2024-07-31 12:24:58,046 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-07-31 12:24:58,134 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-07-31 12:24:58,420 - distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
2024-07-31 12:24:58,532 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-07-31 12:24:59,141 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-07-31 12:24:59,212 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-07-31 12:25:04,185 - distributed.utils_perf - WARNING - full garbage collections took

In [28]:
# results=dask.compute(results)

2024-07-31 12:24:05,647 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)


In [38]:
print(compute_results)

({'QCD_Pt_1000to1400': {'ExampleHistogram': Hist(
  Regular(36, 0, 252, name='msoftdrop', label='Jet msoftdrop'),
  Regular(10, 0, 1, name='n2', label='Jet n2'),
  storage=Weight()) # Sum: WeightedSum(value=193185, variance=193185) (WeightedSum(value=213977, variance=213977) with flow), 'ExampleHistogram1': Hist(
  Regular(10, 0, 1, name='n2b', label='Jet n2b'),
  Regular(36, 0, 252, name='msoftdrop1', label='Jet msoftdrop'),
  storage=Weight()) # Sum: WeightedSum(value=197934, variance=197934) (WeightedSum(value=213977, variance=213977) with flow), 'ExampleHistogram2': Hist(
  Regular(10, 0, 1, name='n2b2', label='Jet n2b2'),
  Regular(36, 0, 252, name='msoftdrop2', label='Jet msoftdrop'),
  storage=Weight()) # Sum: WeightedSum(value=197535, variance=197535) (WeightedSum(value=213977, variance=213977) with flow), 'ExampleHistogram3': Hist(
  Regular(10, 0, 1, name='n2b3', label='Jet n2b3'),
  Regular(36, 0, 252, name='msoftdrop3', label='Jet msoftdrop'),
  storage=Weight()) # Sum: Wei

In [30]:
h=compute_results[0]["QCD_Pt_1000to1400"]["ExampleHistogram"]

In [31]:
print(h)

Hist(
  Regular(36, 0, 252, name='msoftdrop', label='Jet msoftdrop'),
  Regular(10, 0, 1, name='n2', label='Jet n2'),
  storage=Weight()) # Sum: WeightedSum(value=19934, variance=19934) (WeightedSum(value=21996, variance=21996) with flow)


In [32]:
# print(type(h))

In [33]:
h1= compute_results[0]["GluGluHToBB"]["ExampleHistogram1"]

KeyError: 'GluGluHToBB'

In [ ]:
h2= compute_results[0]["GluGluHToBB"]["ExampleHistogram2"]

In [ ]:
h3= compute_results[0]["GluGluHToBB"]["ExampleHistogram3"]

In [ ]:
h4= compute_results[0]["GluGluHToBB"]["ExampleHistogram4"]

In [ ]:
h5= compute_results[0]["GluGluHToBB"]["ExampleHistogram5"]

In [ ]:
h[{'eta':sum, 'pt':sum, 'msoftdrop':sum}].plot1d(label="beta=0")
h1[{'msoftdrop1':sum}].plot1d(label="beta=1")
h2[{'msoftdrop2':sum}].plot1d(label="beta=2")
h3[{'msoftdrop3':sum}].plot1d(label="beta=-0.5")
plt.suptitle('ggF 2018')
plt.ylabel('Events')

plt.legend()


In [ ]:
h[{'eta':sum, 'pt':sum, 'n2':sum}].plot1d(label="beta=0")
h1[{'n2b':sum}].plot1d(label="beta=1")
h2[{'n2b2':sum}].plot1d(label="beta=2")
h3[{'n2b3':sum}].plot1d(label="beta=-0.5")
plt.suptitle('ggF 2018')
plt.ylabel('Events')

plt.legend()

In [ ]:
h[{'eta':sum, 'pt':sum, 'msoftdrop':sum}].plot1d(label="zcut=0.10")
h4[{'msoftdropz1':sum}].plot1d(label="zcut=0.05")
h5[{'msoftdropz2':sum}].plot1d(label="zcut=0.20")
plt.suptitle('ggF 2018')
plt.ylabel('Events')

plt.legend()

In [ ]:
h[{'eta':sum, 'pt':sum, 'n2':sum}].plot1d(label="zcut=0.10")
h4[{'n2z1':sum}].plot1d(label="zcut=0.05")
h5[{'n2z2':sum}].plot1d(label="zcut=0.20")
plt.ylabel('Events')


plt.legend()

In [ ]:
client.close

In [ ]:
client.close()

In [ ]:
# from coffea.analysis_tools import Weights

In [ ]:
# Weights?

In [ ]:
# def analysis(df):
#     processor_instance = msdProcessor(isMC=True)
#     result = processor_instance.process(df)
#     return result

In [ ]:
# df = compute

In [ ]:
# results = dask.delayed(analysis)(df)
# final_result = dask.compute(results)

In [ ]:
#result_compute

In [ ]:
tg_result = dask.compute(tg)
rep_result = dask.compute(rep)